In [ ]:
# 1. Load dataset
import pandas as pd
df = pd.read_csv('AI_Human.csv')
human_texts = df[df['label'] == 'human']['text']
ai_texts = df[df['label'] == 'ai']['text']

In [ ]:
# 2. Split data
from sklearn.model_selection import train_test_split
train_human, val_human = train_test_split(human_texts, test_size=0.2, random_state=42)
train_ai, val_ai = train_test_split(ai_texts, test_size=0.2, random_state=42)

In [ ]:
# 3. Train model
from Models.NgramModel import NgramModel
model = NgramModel(n=3)
model.train(train_human)

In [ ]:
# 4. Score examples
scores_human = [model.score(t) for t in val_human]
scores_ai = [model.score(t) for t in val_ai]

In [ ]:
# 5. Find optimal threshold
import numpy as np
from sklearn.metrics import roc_curve, accuracy_score

labels = [1]*len(scores_human) + [0]*len(scores_ai)
scores = scores_human + scores_ai
fpr, tpr, thresholds = roc_curve(labels, [-s for s in scores])  # negate because lower = more human
best_threshold = thresholds[np.argmax(tpr - fpr)]

In [ ]:
# 6. Evaluate
preds = [model.classify(t, best_threshold) for t in val_human] + \
        [model.classify(t, best_threshold) for t in val_ai]
true = ["human"] * len(val_human) + ["ai"] * len(val_ai)
print("Accuracy:", accuracy_score(true, preds))

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Convert labels to binary format: "human" = 1, "ai" = 0
y_true = [1] * len(val_human) + [0] * len(val_ai)
y_pred = [1 if label == "human" else 0 for label in preds]

print("Precision:", precision_score(y_true, y_pred))
print("Recall:", recall_score(y_true, y_pred))
print("F1 Score:", f1_score(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

In [ ]:
# Combine with text data
val_texts = list(val_human) + list(val_ai)

false_positives = [text for i, text in enumerate(val_texts) if y_true[i] == 0 and y_pred[i] == 1]
false_negatives = [text for i, text in enumerate(val_texts) if y_true[i] == 1 and y_pred[i] == 0]

print(f"False Positives (AI as Human): {len(false_positives)}")
print(f"False Negatives (Human as AI): {len(false_negatives)}")

# Show a few misclassified examples
print("\nExample False Positive:")
print(false_positives[0][:500])  # Show first 500 chars
print("\nExample False Negative:")
print(false_negatives[0][:500])

In [ ]:
import matplotlib.pyplot as plt

plt.hist(scores_human, bins=30, alpha=0.5, label='Human', color='blue')
plt.hist(scores_ai, bins=30, alpha=0.5, label='AI', color='red')
plt.axvline(best_threshold, color='black', linestyle='--', label='Threshold')
plt.title("N-gram Model Score Distribution")
plt.xlabel("Average Negative Log Probability")
plt.ylabel("Count")
plt.legend()
plt.show()